# Technical Analysis
Roboquant comes out of box with support for 200 technical indicators. These are indicators ranging from indicators such as ADX, MACD, RSI, Stochastic, Bollinger Bands to Candlestick pattern recognition. The underlying library that is used is the excellent TA-Lib. TA-Lib can be difficult to use due to its API. But roboquant makes it easy to use it in two ways: 

1. Wraps the indicators with a convenient layer that removes much of the boile plate. However normally you don't have to use layer this directly, but you can if you want to (see TALib and TALibBatch in the package org.roboquant.strategiees.ta) 
2. Have a dedicated TAStrategy (Technical Analysis Strategy) that allows to use the indicators to generate BUY and SELL signals with just a few lines of code

In this notebook we explore the second option

In [ ]:
%use roboquant(0.9-SNAPSHOT)
Welcome()

# Define the strategy
We start by defining the actual strategy we want to use. There are three essestial steps:

1) Create an instance of TAStrategy  and define how much history it should maintain. If you choose this value too low, certain indicators will fail and an exception will be thrown. For example if you want a moving average over 11 days, the history should be at least 11. If you choose it too large, it will take longer before your strategy starts to generate signals since it will wait untill it hjas enough history. 

    For many of the technical indicators, 15 is sufficient (also the default value).

2) Define what you logic to use for generating BUY signals. This can be a combination of indicators and you can also include regular Kotlin code. The only restriction is that the last statement is a boolean; true if you want to BUY, false otherwise.

3) Define what you logic to use for generating SELL signals. The same rules and conditions apply as for the BUY method.

Please note, that although not common practice, the framework doesn't stop you from using bullish indicators to generate a SELL signal and bearish indicators to generate a BUY signal. 

In [ ]:
// Define a strategy with maximum of 15 days of history. It is important that these are enough days to calculate the various indicators.
val strategy = TAStrategy(15)

// We want to generate a BUY signal if we detect either one of the following wto candlestick patternsL: Morningstar or 3WhiteSoldiers
strategy.buy {
    ta.cdlMorningStar(it) || ta.cdl3WhiteSoldiers(it)
}

// We want to generate a SELL signal if we detect the candlestick pattern Two Crows and also observe that the 5 day weighted average is below the 9 day weighted average 
strategy.sell {
    ta.cdl2Crows(it) && ta.ema(it.close, 5) < ta.ema(it.close, 9)
}


# Create and run the back test
Now we can use this strategy just like any other strategy. In this case we use the S&P500 feed.

In [ ]:
val roboquant =  Roboquant(strategy, AccountSummary())

val feed = AvroFeed.sp500()
roboquant.run(feed)

# Evaluate the results
Now lets see what the outcome is. Spoiler alert, this strategy won't make you rich ;) 

In [ ]:
val account = roboquant.broker.account
account.summary()

In [ ]:
account.portfolio.summary()

Let find out which trade generated most profit (realized PnL) and plot the price and all the trades for that asset

In [ ]:
val asset = account.trades.maxByOrNull { it.pnl.value }!!.asset
PriceBarChart(feed, asset, account.trades)

In [ ]:
val metricData = roboquant.logger.getMetric("account.equity")
MetricChart(metricData)